In [87]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
from scipy.spatial import distance
import math


In [88]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, func=None):
        if (func):
            return np.c_[np.ones(self.get_n()), func(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [89]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def confusion_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)

def get_metrics(n_folds):
    return {
        "accuracy": np.zeros(n_folds),
        "precision": np.zeros(n_folds),
        "recall": np.zeros(n_folds),
        "f1_score": np.zeros(n_folds)
    }

def calculate_metrics(metrics, y_test, y_pred):
    metrics["accuracy"][i] = (accuracy(y_test, y_pred))
    metrics["precision"][i] = (precision(y_test, y_pred))
    metrics["recall"][i] = (recall(y_test, y_pred))
    metrics["f1_score"][i] = (f1_score(y_test, y_pred))

def print_metrics(metrics, name, n_folds):
    print("%i-fold cross validation com %s" % (n_folds, name))
    print("acurácia: %.8f +/- %.8f" % (metrics["accuracy"].mean(), metrics["accuracy"].std()))
    print("revocação: %.8f +/- %.8f" % (metrics["precision"].mean(), metrics["precision"].std()))
    print("precisão: %.8f +/- %.8f" % (metrics["recall"].mean(), metrics["recall"].std()))
    print("f1-score: %.8f +/- %.8f" % (metrics["f1_score"].mean(), metrics["f1_score"].std()))
    print("")


In [90]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split

    returns (i, train, test)
    """
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results

# Questão 1

Considere o conjunto de dados disponível em **concrete.csv**, organizado em 9 colunas, sendo as 8 primeiras colunas os atributos e a última coluna a saída. Os 8 atributos referem-se à caracterização de diferentes tipos de concreto para construção civil. A saída é a resistência à compressão do concreto (em megapascals, MPa). Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/d/4353.

In [91]:
data = np.genfromtxt('concrete.csv', delimiter=',')
np.random.seed(666)
folds = k_fold_split(data, 10)
n_folds = len(folds)
features = np.arange(8)
output = 8

a) Considere um modelo de regressão não linear baseado em redes neurais artificiais. Separe os dados aleatoriamente em treino, validação e teste (por exemplo, 60%, 20% e 20%). Nesse cenário, treine e avalie o modelo abaixo:

- **MLP (multilayer perceptron)**: 1 camada oculta e treinamento em *minibatch* via gradiente descendente estocástico com termo de *momentum*. Utilize o conjunto de validação para a justar os hiperparâmetros.

In [92]:
data = Set(data, features, output)
train_X, test_validation_X, train_y, test_validation_y = model_selection.train_test_split(data.get_x(), data.get_y(), test_size=0.4, random_state=666, shuffle=True)
test_X, validation_X, test_y, validation_y = model_selection.train_test_split(test_validation_X, test_validation_y, test_size=0.5, random_state=666)

def multilayer_perceptron(X, y):
    pass

(618, 8)
(618,)
(206, 8)
(206,)
(206, 8)
(206,)


b) Apresente as curvas da função custo nos conjuntos de treinamento e validação ao longo das épocas. Reporte também para os conjuntos de treino, validação e teste as métricas abaixo:
- **RMSE (root mean squared error)**: 
- **MAE (mean absolute error)**:
- **MRE (mean relative error)**:


## Questão 2

Considere o conjunto de dados disponível em **vowel.csv**, organizado em 11 colunas, sendo as 10 primeiras colunas os atributos e a última coluna a saída. Os 10 atributos referem-se à caracterização de amostras da fala de britânicos. A saída é o fonema de vogal correspondente, dentre as 11 possibilidades. Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/d/307.

In [93]:
data = np.genfromtxt('vowel.csv', delimiter=',')
np.random.seed(666)
folds = k_fold_split(data, 10)
n_folds = len(folds)
features = np.arange(11)
output = 11

a) Considere um mo delo de classificação não linear baseado em redes neurais artificiais. Separe os dados aleatoriamente em treino, validação e teste (por exemplo, 60%, 20% e 20%). Nesse cenário, treine e avalie o modelo abaixo:

- **MLP (multilayer perceptron)**: 1 camada o culta e treinamento em *minibatch* via gradiente descendente esto cástico com termo de *momentum*. Utilize o conjunto de validação para a justar os hiperparâmetros.

b) Apresente as curvas da função custo nos conjuntos de treinamento e validação ao longo das épocas. Reporte também a acurácia obtida para os conjuntos de treino, validação e teste.